# Data preprocessing: Numbeo Dataset

In [1]:
import pandas as pd
import requests
from tqdm import tqdm

In [2]:
api_key = 'wl1rvko3ziykgx'

## Creating Tables

### 1. countries.csv

In [4]:
countries_eu = ['Austria', 'Belgium', 'Bulgaria', 'Croatia', 'Cyprus', 'Czech Republic', 'Denmark', 'Estonia',
                'Finland', 'France', 'Germany', 'Greece', 'Hungary', 'Ireland', 'Italy', 'Latvia', 'Lithuania',
                'Luxembourg', 'Malta', 'Netherlands', 'Poland', 'Portugal', 'Romania', 'Slovakia', 'Slovenia',
                'Spain', 'Sweden']

# Create a dataframe with column: country
df_countries = pd.DataFrame(countries_eu, columns=['country'])

# Create a column with corresponding flag image
# For each country, get the image from the folder 'img/flags'
df_countries['flag'] = df_countries['country'].apply(lambda x: f'./img/flags/{x}.png')

# Export df_countries to csv
df_countries.to_csv('./data/countries.csv', index=False)

### 2. country_prices.csv

In [5]:
dic_country_prices = []
desired_items = [1, 18, 26, 27, 30, 33, 105, 40, 34]

for country in tqdm(countries_eu, desc='countries'):
    country_prices = pd.DataFrame.from_dict(requests.get('https://www.numbeo.com/api/country_prices?api_key=' + api_key + '&country=' + country + '&currency=EUR').json())
    for row in country_prices.iterrows():
        if row[1]['prices']['item_id'] in desired_items:
            new_data = {}
            new_data['country'] = country
            new_data['currency'] = row[1]['currency']
            new_data['yearLastUpdate'] = row[1]['yearLastUpdate']
            new_data['monthLastUpdate'] = row[1]['monthLastUpdate']
            new_data['contributors'] = row[1]['contributors']
            new_data['item_id'] = row[1]['prices']['item_id']
            new_data['item_name'] = row[1]['prices']['item_name']
            new_data['average_price'] = row[1]['prices']['average_price']
            try:
                new_data['lowest_price'] = row[1]['prices']['lowest_price']
            except:
                new_data['lowest_price'] = None
            try: 
                new_data['highest_price'] = row[1]['prices']['highest_price']
            except:
                new_data['highest_price'] = None
            dic_country_prices.append(new_data)

df_country_prices = pd.DataFrame.from_dict(dic_country_prices)
df_country_prices['lowest_price'] = df_country_prices['lowest_price'].fillna(df_country_prices['average_price'])
df_country_prices['highest_price'] = df_country_prices['highest_price'].fillna(df_country_prices['average_price'])

df_country_prices.to_csv('./data/country_prices.csv', index=False)

countries:   0%|          | 0/27 [00:00<?, ?it/s]

countries: 100%|██████████| 27/27 [00:40<00:00,  1.51s/it]


### 3. country_indices.csv

In [5]:
dic_country_indices = []

for country in tqdm(countries_eu, desc='countries'):
    country_indices = requests.get('https://www.numbeo.com/api/country_indices?api_key=' + api_key + '&country=' + country).json()
    dic_country_indices.append(country_indices)

df_country_indices = pd.DataFrame.from_dict(dic_country_indices)
df_country_indices.rename(columns={'name': 'country'}, inplace=True)
df_country_indices = df_country_indices[['country', 'quality_of_life_index', 'purchasing_power_incl_rent_index', 'rent_index', 'health_care_index', 'safety_index']]

# Normalize column 'quality_of_life_index' to a scale from 0 to 100 (previously 50 to 100)
min_value = df_country_indices['quality_of_life_index'].min()
max_value = df_country_indices['quality_of_life_index'].max()
df_country_indices ['quality_of_life_index'] = ((df_country_indices['quality_of_life_index'] - min_value) / (max_value - min_value)) * (100 - 0) + 0

# Normalize column 'purchasing_power_incl_rent_index' to a scale from 0 to 100, but inversing the values (previously 50 to 100)
min_value = df_country_indices['purchasing_power_incl_rent_index'].min()
max_value = df_country_indices['purchasing_power_incl_rent_index'].max()
df_country_indices['purchasing_power_incl_rent_index'] = 1 - ((df_country_indices['purchasing_power_incl_rent_index'] - min_value) / (max_value - min_value))
df_country_indices['purchasing_power_incl_rent_index'] = df_country_indices['purchasing_power_incl_rent_index'] * (100 - 0) + 0

# Normalize column 'rent_index' to a scale from 0 to 100, but inversing the values (previously 25 to 100)
min_value = df_country_indices['rent_index'].min()
max_value = df_country_indices['rent_index'].max()
df_country_indices['rent_index'] = 1 - ((df_country_indices['rent_index'] - min_value) / (max_value - min_value))
df_country_indices['rent_index'] = df_country_indices['rent_index'] * (100 - 0) + 0

# Normalize column 'health_care_index' to a scale from 0 to 100 (previously 60 to 100)
min_value = df_country_indices['health_care_index'].min()
max_value = df_country_indices['health_care_index'].max()
df_country_indices ['health_care_index'] = ((df_country_indices['health_care_index'] - min_value) / (max_value - min_value)) * (100 - 0) + 0

# Normalize column 'safety_index' to a scale from 0 to 100 (previously 60 to 100)
min_value = df_country_indices['safety_index'].min()
max_value = df_country_indices['safety_index'].max()
df_country_indices ['safety_index'] = ((df_country_indices['safety_index'] - min_value) / (max_value - min_value)) * (100 - 0) + 0

df_country_indices.to_csv('./data/country_indices.csv', index=False)

countries:   0%|          | 0/27 [00:00<?, ?it/s]

countries: 100%|██████████| 27/27 [00:49<00:00,  1.82s/it]


In [ ]:
'''
For more about each index, see:

quality_of_life_index -> https://www.numbeo.com/quality-of-life/indices_explained.jsp

purchasing_power_incl_rent_index and rent_index -> https://www.numbeo.com/cost-of-living/cpi_explained.jsp

health_care_index -> https://www.numbeo.com/health-care/indices_explained.jsp

safety_index -> https://www.numbeo.com/crime/indices_explained.jsp

'''